In [ ]:
'''ResNet in PyTorch.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''

In [ ]:
"""
pre-trained weights:
https://drive.google.com/file/d/1K5zBBxYRzFDqPQqGQ15Lo4vjrexwVtM1/view
"""

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [3]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=512):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.fc_bn = nn.BatchNorm1d(512)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 7)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = self.fc_bn(out)
        return out

In [5]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [12]:
def test():
    net = ResNet18()
    y = net(Variable(torch.randn(32,3,112,112)))
    print(y.size())

In [13]:
test()

torch.Size([32, 512])


In [14]:
a = ResNet18()
a

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [15]:
torch.load('resnet18_scale_fold0_best.pth.tar', map_location=torch.device('cpu'))

{'best_mse': 0.0004508470387135886,
 'epoch': 25,
 'optimizer': {'state': {140432399631768: {'exp_avg_sq': tensor([1.6095e-12, 1.6330e-12, 1.6130e-12, 1.2002e-12, 1.5578e-12, 1.1830e-12,
            1.6173e-12, 2.2244e-12, 2.0742e-12, 2.0866e-12, 1.7451e-12, 1.9588e-12,
            1.7440e-12, 1.5696e-12, 1.8236e-12, 1.5253e-12, 1.7879e-12, 2.1534e-12,
            2.8808e-12, 1.3203e-12, 1.0310e-12, 1.8671e-12, 2.0330e-12, 1.3909e-12,
            2.0614e-12, 1.6500e-12, 1.5782e-12, 1.8403e-12, 2.1546e-12, 2.6610e-12,
            1.4508e-12, 1.3575e-12, 2.2089e-12, 2.6984e-12, 1.3306e-12, 7.5076e-13,
            1.8345e-12, 1.4617e-12, 1.5361e-12, 2.0091e-12, 2.0930e-12, 1.5170e-12,
            2.0140e-12, 1.8684e-12, 1.2723e-12, 3.0368e-12, 1.7577e-12, 1.8523e-12,
            1.5221e-12, 1.6442e-12, 1.6374e-12, 1.1612e-12, 1.9032e-12, 1.2758e-12,
            1.4141e-12, 1.7469e-12, 2.0781e-12, 2.8842e-12, 1.6585e-12, 1.9731e-12,
            9.9723e-13, 1.6092e-12, 1.4351e-12, 1.8030e-1

In [ ]:
torch.load('tensors.pt', map_location=torch.device('cpu'))

In [ ]:
torch.load('tensors.pt', map_location=lambda storage, loc: storage)